Auteurs : Hachemin Pierre-Yves et Seys Thibaut

Date : 25/04/2018

Cours : IS3024AB - Data Mining

# Etude sur le transport en Ile-de-France

* [Introduction](#Introduction)

    + [Contexte et objectifs](#Contexte-et-objectifs)
    
    + [Requirements](#Requirements)

* [Datasets](#Datasets)

    + [Description des datasets](#Description-des-datasets)
    
    + [Composition des datatsets](#Composition-des-datasets)
    
* [Traitements des datasets](#Traitements-des-datasets)

    + [Datasets de validation](#Datasets-de-validation)

## Introduction

### Contexte et objectifs

Pour notre projet de Data Mining, nous avons choisi d'explorer les données issues de la palteforme Open Data du Syndicat des Transports d'Ile-de-France (STIF) disponible à [cette adresse](https://opendata.stif.info/). Nous avons décidé d'explorer les données issues de la validation des voyageurs sur les différents réseaux ferrés d'Ile-de-France.

Notre objectif est de mener une double étude sur les caractéristiques actuelles du réseau ferré et de son efficacité. Nous allons donc d'une part appliquer des algorithmes de clustering sur les différentes stations du réseau afin de mettre en évidence les profils types. Nous allons d'autre part essayer de recréer des lignes de transports reliant les différentes stations et comparer les résultats obtenus au réseau actuellement en place.

### Requirements

Pour que notre projet fonctionne correctement, nous avons besoin d'importer les libraries suivantes :

In [1]:
import csv
import folium
import pandas as pd
import numpy as np

## Datasets

### Description des datasets

Parmis les jeux de données mis à disposition par la plateforme Open Data du STIF, nous avons choisi d'utiliser les 4 types de datasets suivants :

- [validations-nombre-par-jour-2017s1](https://opendata.stif.info/explore/dataset/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-sem/) et [validations-nombre-par-jour-2017s2](https://opendata.stif.info/explore/dataset/validations-sur-le-reseau-ferre-profils-horaires-par-jour-type-2e-sem/) : contiennent le nombre de validations par jour par station par catégorie de titre de transport pour chaqeu semestre de 2017.

- [validations-profils-horaires-2017s1](https://opendata.stif.info/explore/dataset/validations-sur-le-reseau-ferre-profils-horaires-par-jour-type-1er-sem/) et [validations-profils-horaires-2017s2](https://opendata.stif.info/explore/dataset/validations-sur-le-reseau-ferre-profils-horaires-par-jour-type-2e-sem/) : contiennent les pourcentages moyens de validation par tranche horaire à la station donnée suivant le type de journée (jour ordinaire, dimanche, etc...)  pour chaque semestre de 2017.

- [emplacement-des-gares-idf](https://opendata.stif.info/explore/dataset/emplacement-des-gares-idf/) : contient les emplacements de l'ensemble des stations du réseau ferré d'Ile-de-France.

- [referentiel-arret-tc-idf-](https://opendata.stif.info/explore/dataset/referentiel-arret-tc-idf/) : recense l'ensemble des stations des réseaux de surface et ferré de transport d'Ile-de-France suivant différents niveaux de granularités.

Tous ces datasets sont disponibles dans le dossier `Data` sous forme de fichiers csv.

### Composition des datasets

Nous allons décrire dans cette partie les différents champs que nous allons utiliser pour chaque type de datasets que nous utilisons. De la documentation plus précise émise par le STIF se trouve dans le dossier `doc` sous forme de fichiers PDF.

#### **Datasets du nombre de validations par jour**

Les champs que nous avons choisi de garder sont représentés dans le tableau suivant :

| Nom du champs | Description | Valeur |
|---------------|-------------|--------|
| LIBELLE_ARRET | Libellée de l'arrêt | string |
| ID_REFA_LDA | Id de l'arrêt | float |
| JOUR | Jour de l'année | string |
| CATEGORIE_TITRE | Catégorie du titre | AMETHYSTE, AUTRE TITRE, FGT, IMAGINE R, NAVIGO, TST |
| NB_VALD | Nombre de validations | int ou 'Moins de 5' |

#### **Datasets du profil de validation**

Les champs que nous avons choisi de garder sont représentés dans le tableau suivant :

| Nom du champs | Description | Valeur |
|---------------|-------------|--------|
| LIBELLE_ARRET | Libellée de l'arrêt | string |
| ID_REFA_LDA | Id de l'arrêt | float |
| CAT_JOUR | Type de journée | JOHV, SAHV, JOVS, SAVS, DIJFP |
| TRNC_HORR_60 | Tranche horaire | Interval (ex: 13H - 14H) |
| pourc_validations | Pourcentage de validations | float |

#### **Datasets du référentiels des stations du réseau**

La modélisation d'un arrêt est une chose compliquée et le STIF possède trois niveaux de détails décrit ci-après du niveau le plus large au moins large :

1. LDA : le premier niveau concerne l'ensemble des arrêts du même nom. Par exemple, pour l'arrêt 'République' il y a à la fois les différentes lignes de métro et les arrêts de bus.

2. ZDL : le second niveau décrit l'ensemble des arrêts d'un même mode de transport. En reprenant l'exemple de l'arrêt 'République', on aura une ZDL pour le métro et une autre pour les arrêts de bus.

3. ZDE : le dernier niveau décrit une zone d'embarquement. C'est l'endroit où l'usager peut physiquement emprunter la ligne de transport. Dans le cas de la ZDL des métro à 'République', il y aura une ZDE par ligne de métro.

Ce dataset nous permet de faire les liens entre les stations des datasets de validation qui contiennet l'id de la LDA et le dataset des emplacements des stations qui contiennent les id de ZDE et ZDL. Voici les champs que nous allons utiliser au cours de notre traitement de données :

| Nom du champs | Description | Valeur |
|---------------|-------------|--------|
| ZDEr_LIBELLE_TYPE_ARRET | Libellée du type d'arrêt | Arrêt de bus, Station ferrée / Val, Arrêt de tram, Station de métro, Station de funiculaire |
| LDA_ID_REF_A | Id de la LDA | float |
| ZDLr_ID_REF_A | Id de la ZDL | float |
| ZDEr_ID_REF_A | Id de la ZDE | float |

#### **Datasets des emplacements des stations du réseau ferré**

Ce dataset va nous permettre de relier les données des datasets de validations à des positions géographiques et aux lignes du réseau ferré passant par ces stations. Les champs dont nous allons avoir besoin sont décrits dans le tableau suivant :

| Nom du champs | Description | Valeur |
|---------------|-------------|--------|
| Geo Point | Coordonnées de la station | tuple |
| LIGNES | Ligne passant par cette station | string |
| ID_REF_ZDL | Id de la ZDL | float |
| ID_REF_ZDE | Id de la ZDE | float |

## Traitements des datasets

L'objectif de cette partie est d'arriver à un unique dataset sur lequel on effectuera nos calculs. Pour cela nous allons partir du dataset du nombre de validation par jour par station et le transformer pour avoir comme colonne : 

- JOUR : date du jour

- LIBELLE_ARRET : Libellée de l'arrêt

- ID_REFA_LDA : ID LDA de l'arrêt

- LIGNES : Lignes passant par cette station

- CAT_JOUR : Catégorie du jour parmi JOHV, SAHV, JOVS, SAVS et DIJFP

- COORDINATE_X : Longitude de la station

- COORDINATE_Y : Latitude de la station

- TRANCHE_HORRAIRE (x24) : Une colonne par pourcentage de validation de la tranche horaire

- NB_VALIDATION (x7) : Une colonne par titre de transport contenant le nombre de validation

On arrivera donc à 38 colonnes par station par jour de l'année 2017.

### Datasets de validation

Pour commencer nous allons charger les datasets du nombre de valdiation et des profiles horaires. Pour les datasets de profils horaires, le STIF nous fournit une ligne par arrêt par catégorie de jour par tranche horraire. Or, nous souhaitons obtenir une ligne par arrêt et par catégorie de jour, les pourcentages suivants la tranche horaire étant contenus dans une colonne par tranche. Pour le dataset du nombre de validation par jour, nous avons exactement le même problème avec les catégories de titres de transport. Nous allons donc effectuer cette transformation au moment où nous chargeons les données dans des DataFrames pandas.

In [5]:
# Fonction générique permettant le chargement des csv de validations
def load_csv(csv_filename, columns, to_flatten, value):
    with open(csv_filename, 'r', newline='\n') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        
        data = {}
        header = {elt: index for index, elt in enumerate(next(csv_reader))}
        flattens = set()
        
        for row in csv_reader:
            key = tuple(row[header[column]] for column in columns)
            flattens.add(row[header[to_flatten]])
            try:
                data[key][row[header[to_flatten]]] = row[header[value]]
            except KeyError:
                data[key] = {row[header[to_flatten]]: row[header[value]]}
                
        df_dict = {elt:  [] for elt in columns}
        df_dict.update({elt: [] for elt in flattens})
    
        for key, values in data.items():
            for index, column in enumerate(columns):
                df_dict[column].append(key[index])
            for flatten in flattens:
                df_dict[flatten].append(values.get(flatten, 0)) 
        
        return pd.DataFrame(df_dict)

In [6]:
# Chargement des différentes DataFrame
p_columns = ['LIBELLE_ARRET', 'CAT_JOUR', 'ID_REFA_LDA']
p_flatten = 'TRNC_HORR_60'
p_value = 'pourc_validations'

v_columns = ['LIBELLE_ARRET', 'JOUR', 'ID_REFA_LDA']
v_flatten = 'CATEGORIE_TITRE'
v_value = 'NB_VALD'

profile_s1 = load_csv('Data/validations-profils-horaires-2017s1.csv', p_columns, p_flatten, p_value)
profile_s2 = load_csv('Data/validations-profils-horaires-2017s2.csv', p_columns, p_flatten, p_value)

validation_s1 = load_csv('Data/validations-nombre-par-jour-2017s1.csv', v_columns, v_flatten, v_value)
validation_s2 = load_csv('Data/validations-nombre-par-jour-2017s2.csv', v_columns, v_flatten, v_value)

Nous allons ensuite finir le chargement de ces datasets en réalisant une suite de pré-traitements comme supprimer les colonnes 'NON DEFINI' et les lignes contenant un ID_REFA_LDA nul.

In [12]:
profile_s1 = profile_s1.drop('ND', axis=1)
profile_s1 = profile_s1.drop(profile_s1[profile_s1['ID_REFA_LDA'] == ''].index)

profile_s2 = profile_s2.drop('ND', axis=1)
profile_s2 = profile_s2.drop(profile_s2[profile_s2['ID_REFA_LDA'] == ''].index)

validation_s1 = validation_s1.drop('NON DEFINI', axis=1)
validation_s1 = validation_s1.drop(validation_s1[validation_s1['ID_REFA_LDA'] == ''].index)

validation_s2 = validation_s2.drop('NON DEFINI', axis=1)
validation_s2 = validation_s2.drop(validation_s2[validation_s2['ID_REFA_LDA'] == ''].index)

### Emplacement des stations

### Dataset final

### Fonctions utilitaires